In [1]:
import os
import pickle
import argparse
import unicodedata
import pandas as pd

# Import Tokenizer
from transformers import AutoTokenizer
from kobert_tokenizer import KoBERTTokenizer

In [2]:
def define_argparser():
    p = argparse.ArgumentParser()
    # skt/kobert-base-v1
    p.add_argument("--pretrained_model_name", default ='klue/roberta-base', help="Pre-trained model name to be going to train. Tokenizer will be assigned based on the model.")
    p.add_argument("--load_fn", default='data/dataset/test.pickle', help="Original data to be going to preprocess.")
    p.add_argument("--save_path", default='data/encoded', help="Original data to be going to preprocess.")
    p.add_argument("--with_text", action="store_true", help="Return values with text for inference.")

    config = p.parse_args(args=[])

    return config

In [3]:
config = define_argparser()

In [4]:
USE_KOBERT = True if config.pretrained_model_name == 'skt/kobert-base-v1' else False
data = pd.read_pickle(config.load_fn)
texts = data['sentence'].values.tolist()
nes = data['ne'].values.tolist()

In [5]:
data

,sentence,ne,source,sentence_class
192,제관원으로 각 입사하여 피고 양회 동해공장 기계과 소속으로 시멘트 제조기계의 보수 ...,"{1: {'form': '제관원', 'label': 'JB', 'begin': 0,...",NaN,JB
18,1. 원고와의 사이에 피고의 정관이 정하는 바에 따라 위 같은 날부터 1988.2....,"{1: {'form': '원고', 'label': 'PS_VICTIM', 'begi...",NaN,JB
134,"'시설'이 양식생물을 의미하는 것과는 달리, 철주, 표지목과 같은 어구시설물을 의미...",{},NaN,Out
136,"자로 전면 개정되기 전의 것, 이하 같다) 제75조 제3항, 동법시행령 제72조 제...",{},NaN,Out
70,"양정산의 소유로 사정된 것으로 임야대장에 등재되어 있는 사실, 한편 원고 양순희의 ...","{1: {'form': '양정산', 'label': 'NM', 'begin': 0,...",NaN,NM
...,...,...,...,...
12,위 (명칭 생략)대학 교원인사위원회에 법학과 학생들과의 관계개선과 교수의 품위를 지...,"{1: {'form': '교수', 'label': 'JB', 'begin': 37,...",NaN,JB
157,있어 결국 어업권의 평가에 있어 양식생물의 피해까지도 고려하여 평가하도록 규정하고 ...,{},NaN,Out
14,"소외 8의 각 일부 증언 및 원심에서의 원고본인신문결과에 변론의 전취지를 종합하면,...","{1: {'form': '소외 8', 'label': 'RL', 'begin': 0...",NaN,RL
181,"한편, 개심술 후의 뇌손상의 원인으로는 색전증과 뇌허혈이 양대원인을 이루고 있는데 ...","{1: {'form': '뇌손상', 'label': 'PC', 'begin': 11...",NaN,BP


In [6]:
texts

['제관원으로 각 입사하여 피고 양회 동해공장 기계과 소속으로 시멘트 제조기계의 보수 등 업무에 종사하던 중 1981.3.경 피고 양회 계열그룹의 경영방침에 따라 위 업무가 같은 계열회사인 피고 쌍용중공업주식회사(이하 피고 중공업이라 한다)로',
 '1. 원고와의 사이에 피고의 정관이 정하는 바에 따라 위 같은 날부터 1988.2.29.까지 4년간 원고를 피고가 설치, 운영하는 숭의여자전문대학의 부교수로 임용한다는 계약을 체결하여 원고가 위 기간동안 위 전문대학 도서관학과 부교수로 재직한',
 "'시설'이 양식생물을 의미하는 것과는 달리, 철주, 표지목과 같은 어구시설물을 의미한다고 풀이함이 상당하다",
 '자로 전면 개정되기 전의 것, 이하 같다) 제75조 제3항, 동법시행령 제72조 제1항 제1호에 의한 어업권평가액 이외에 별도로 양식생물피해액을 평가하여 보상하여야 한다고 주장함에 대하여, 공공사업으로 인하여 어업권이 소멸되는 경우 보상할 손실액의',
 '양정산의 소유로 사정된 것으로 임야대장에 등재되어 있는 사실, 한편 원고 양순희의 아버지이자, 원고 노연주의 시아버지이며, 나머지 원고들의 할아버지인 소외 망 양정산은 원래 위 경기 양주군 회천면 덕계리에서 거주하다가 경기 파주군',
 '청주기계공업고등학교를 졸업하고, 1976.6.16.경부터 그 다음해까지 소외 현대건설주식회사에, 1977.10.28.부터 1978.10.28.까지 대림산업주식회사에, 1979.9.8.부터 1980.9.10.까지 전공으로 동아건설주식회사에 각 취업하여 사우디아라비아 파견근무를 하는 등 국내외에서 주로 전기배관공등으로',
 '있는 경우와 없는경우로 구분하여 그 보상액을 달리하도록 규정하고 있을 뿐 아니라 양식생물이 있는 경우에는 양식생물이 없는 경우보다 일정액을 증액하도록 하고 있어 결국 어업권의 평가에 있어 양식생물의 피해로 인해 손실까지도 고려하도록',
 '피고인 1의 변호인의 항소이유 요지는, 동 피고인이 이 사건 범행에 이르게 된 경위나 이 사건에서 문제가 된 먹장어의

In [7]:
nes

[{1: {'form': '제관원', 'label': 'JB', 'begin': 0, 'end': 3},
  2: {'form': '피고 양회 동해공장 기계과',
   'label': 'PS_ASSAILANT',
   'begin': 13,
   'end': 27},
  3: {'form': '1981.3.경', 'label': 'TI_DATE', 'begin': 59, 'end': 67},
  4: {'form': '피고 양회 계열그룹', 'label': 'PS_ASSAILANT', 'begin': 68, 'end': 78},
  5: {'form': '피고 쌍용중공업주식회사',
   'label': 'PS_ASSAILANT',
   'begin': 104,
   'end': 116},
  6: {'form': '피고 중공업', 'label': 'PS_ASSAILANT', 'begin': 120, 'end': 126}},
 {1: {'form': '원고', 'label': 'PS_VICTIM', 'begin': 3, 'end': 5},
  2: {'form': '피고', 'label': 'PS_ASSAILANT', 'begin': 12, 'end': 14},
  3: {'form': '같은 날부터', 'label': 'TI_DATE', 'begin': 32, 'end': 38},
  4: {'form': '1988.2.29.까지', 'label': 'TI_DATE', 'begin': 39, 'end': 51},
  5: {'form': '4년간', 'label': 'TI_DURATION', 'begin': 52, 'end': 55},
  6: {'form': '원고', 'label': 'PS_VICTIM', 'begin': 56, 'end': 58},
  7: {'form': '피고', 'label': 'PS_ASSAILANT', 'begin': 60, 'end': 62},
  8: {'form': '숭의여자전문대학의 부교수', 'label': 'JB', '

In [8]:
pretrained_model_name = config.pretrained_model_name
print(pretrained_model_name)

klue/roberta-base


In [9]:
if USE_KOBERT:
    tokenizer_loader = KoBERTTokenizer # 해당 안 됨
else:
    tokenizer_loader = AutoTokenizer

tokenizer = tokenizer_loader.from_pretrained(pretrained_model_name)
print("Tokenizer loaded :", tokenizer.name_or_path)

Tokenizer loaded : klue/roberta-base


In [10]:
def get_label_dict(labels):
    BIO_labels = ['O']
    for label in labels:
        BIO_labels.append('B-'+label)
        BIO_labels.append('I-'+label)

    label_to_index = {label: index for index, label in enumerate(BIO_labels)}
    index_to_label = {index: label for index, label in enumerate(BIO_labels)}

    return label_to_index, index_to_label

In [11]:
# label_list = ["PS", "FD", "TR", "AF", "OG", "LC", "CV", "DT", "TI", "QT", "EV", "AM", "PT", "MT", "TM"]
label_list = ["CN", "NC", "RL", "LC", "NM", "GD", "AG", "JB", "PS", "DP", "TI", "MC", "PC", "BP", "QT", "RA"]
label_to_index, index_to_label = get_label_dict(label_list) # 레이블 리스트를 받아서 BIO 매핑을 진행
cls_token = tokenizer.cls_token
sep_token = tokenizer.sep_token
max_length = 512

In [12]:
print(label_to_index)
print(index_to_label)
print(cls_token)
print(sep_token)

{'O': 0, 'B-CN': 1, 'I-CN': 2, 'B-NC': 3, 'I-NC': 4, 'B-RL': 5, 'I-RL': 6, 'B-LC': 7, 'I-LC': 8, 'B-NM': 9, 'I-NM': 10, 'B-GD': 11, 'I-GD': 12, 'B-AG': 13, 'I-AG': 14, 'B-JB': 15, 'I-JB': 16, 'B-PS': 17, 'I-PS': 18, 'B-DP': 19, 'I-DP': 20, 'B-TI': 21, 'I-TI': 22, 'B-MC': 23, 'I-MC': 24, 'B-PC': 25, 'I-PC': 26, 'B-BP': 27, 'I-BP': 28, 'B-QT': 29, 'I-QT': 30, 'B-RA': 31, 'I-RA': 32}
{0: 'O', 1: 'B-CN', 2: 'I-CN', 3: 'B-NC', 4: 'I-NC', 5: 'B-RL', 6: 'I-RL', 7: 'B-LC', 8: 'I-LC', 9: 'B-NM', 10: 'I-NM', 11: 'B-GD', 12: 'I-GD', 13: 'B-AG', 14: 'I-AG', 15: 'B-JB', 16: 'I-JB', 17: 'B-PS', 18: 'I-PS', 19: 'B-DP', 20: 'I-DP', 21: 'B-TI', 22: 'I-TI', 23: 'B-MC', 24: 'I-MC', 25: 'B-PC', 26: 'I-PC', 27: 'B-BP', 28: 'I-BP', 29: 'B-QT', 30: 'I-QT', 31: 'B-RA', 32: 'I-RA'}
[CLS]
[SEP]


In [13]:
if USE_KOBERT:
    encoded = tokenizer(texts,
                        add_special_tokens=True,
                        padding=False,
                        return_attention_mask=True,
                        truncation=True,
                        max_length=max_length)
else:
    encoded = tokenizer(texts,
                        add_special_tokens=True,
                        padding=False,
                        return_attention_mask=True,
                        truncation=True,
                        max_length=max_length,
                        return_offsets_mapping=True)

print("Sentences encoded : |input_ids| %d, |attention_mask| %d" %
        (len(encoded['input_ids']), len((encoded['attention_mask']))))

Sentences encoded : |input_ids| 177, |attention_mask| 177


In [14]:
print(type(encoded))
print(dir(encoded))

<class 'transformers.tokenization_utils_base.BatchEncoding'>
['_MutableMapping__marker', '__abstractmethods__', '__class__', '__contains__', '__copy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_encodings', '_n_sequences', 'char_to_token', 'char_to_word', 'clear', 'convert_to_tensors', 'copy', 'data', 'encodings', 'fromkeys', 'get', 'is_fast', 'items', 'keys', 'n_sequences', 'pop', 'popitem', 'sequence_ids', 'setdefault', 'to', 'token_to_chars', 'token_to_sequence', 'token_to_word', 'tokens', 'update', 'values', 'word_ids', 'word_to_chars', 'word

In [15]:
for i in range (0, 1):
    print(encoded['input_ids'][i])  # 토큰 ID 리스트
    print(encoded['attention_mask'][i])  # 어텐션 마스크
    print(encoded['token_type_ids'][i])  # (있을 경우) 토큰 타입 ID
    print(encoded['offset_mapping'][i])  # 오프셋 매핑

[0, 1545, 2133, 2252, 6233, 544, 7957, 7488, 8305, 30441, 7283, 2086, 2121, 5276, 2145, 4546, 6233, 12390, 4755, 19346, 2079, 4687, 886, 4148, 2170, 10313, 2205, 2414, 1570, 12148, 18, 23, 18, 591, 8305, 30441, 5096, 6941, 2079, 3939, 2239, 2540, 2170, 3653, 1485, 4148, 2116, 555, 2073, 5096, 6166, 2179, 8305, 8643, 2284, 6116, 2223, 2096, 6166, 12, 4547, 8305, 6663, 2052, 2181, 3605, 13, 991, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[(0, 0), (0, 1), (1, 2), (2, 3), (3, 5), (6, 7), (8, 10), (10, 12), (13, 15), (16, 18), (19, 21), (21, 22), (22, 23), (24, 26), (26, 27), (28, 30), (30, 32), (33, 36), (37, 39), (39, 4

In [16]:
df = pd.DataFrame([encoded["input_ids"], encoded["attention_mask"], data['sentence_class'].values],
                  index=["input_ids", "attention_mask", "sentence_class"]).T

In [17]:
df

,input_ids,attention_mask,sentence_class
0,"[0, 1545, 2133, 2252, 6233, 544, 7957, 7488, 8...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",JB
1,"[0, 21, 18, 7577, 2522, 2079, 3734, 2170, 8305...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",JB
2,"[0, 11, 3953, 11, 1504, 6277, 2065, 2266, 2069...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Out
3,"[0, 1517, 2200, 5636, 4615, 2496, 2015, 1537, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Out
4,"[0, 24673, 2066, 2079, 4913, 2200, 5183, 2897,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",NM
...,...,...,...
172,"[0, 1485, 12, 7864, 12607, 13, 3723, 8716, 217...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",JB
173,"[0, 1513, 2051, 3983, 12489, 2207, 2079, 3777,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",Out
174,"[0, 6954, 28, 2079, 544, 3951, 8105, 1116, 149...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",RL
175,"[0, 3995, 16, 558, 2361, 2072, 1943, 2079, 776...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",BP


In [18]:
def get_kobert_offset_mappings(kobert_tokens):
    """
    This function is for KoBERTTokenizer, which cannot return offset_mappings.
    """
    kobert_offset_mappings = []
    offset_begin = -1
    for i in kobert_tokens:
        if i in ["[CLS]", "[SEP]"]:
            kobert_offset_mappings.append((0, 0))
            continue
        if i.startswith('▁'):
            offset_begin += 1
            i = i[1:]
        offset_end = offset_begin+len(i)
        kobert_offset_mappings.append((offset_begin, offset_end))
        offset_begin = offset_end

    return kobert_offset_mappings

In [19]:
ne_ids = []

if USE_KOBERT:
    text_tokens = [[cls_token] + tokenizer.tokenize(text)[:max_length-2] + [sep_token] for text in texts]
    offset_mappings = [get_kobert_offset_mappings(text_token) for text_token in text_tokens]
    text_normalize=True
else: 
    offset_mappings = encoded['offset_mapping']
    text_normalize=False

In [20]:
i = 0

for text, ne, offset_mapping in zip(texts, nes, offset_mappings):
    s = text
    t = tokenizer.tokenize(text)
    n = ne
    om = offset_mapping

    print(s)
    print(t)
    print(om)
    print(n)
    print()

    i += 1

    if i == 3:
        break

제관원으로 각 입사하여 피고 양회 동해공장 기계과 소속으로 시멘트 제조기계의 보수 등 업무에 종사하던 중 1981.3.경 피고 양회 계열그룹의 경영방침에 따라 위 업무가 같은 계열회사인 피고 쌍용중공업주식회사(이하 피고 중공업이라 한다)로
['제', '##관', '##원', '##으로', '각', '입사', '##하여', '피고', '양회', '동해', '##공', '##장', '기계', '##과', '소속', '##으로', '시멘트', '제조', '##기계', '##의', '보수', '등', '업무', '##에', '종사', '##하', '##던', '중', '1981', '.', '3', '.', '경', '피고', '양회', '계열', '##그룹', '##의', '경영', '##방', '##침', '##에', '따라', '위', '업무', '##가', '같', '##은', '계열', '##회사', '##인', '피고', '쌍용', '##중', '##공업', '##주', '##식', '##회사', '(', '이하', '피고', '중공업', '##이', '##라', '한다', ')', '로']
[(0, 0), (0, 1), (1, 2), (2, 3), (3, 5), (6, 7), (8, 10), (10, 12), (13, 15), (16, 18), (19, 21), (21, 22), (22, 23), (24, 26), (26, 27), (28, 30), (30, 32), (33, 36), (37, 39), (39, 41), (41, 42), (43, 45), (46, 47), (48, 50), (50, 51), (52, 54), (54, 55), (55, 56), (57, 58), (59, 63), (63, 64), (64, 65), (65, 66), (66, 67), (68, 70), (71, 73), (74, 76), (76, 78), (78, 79), (80, 82), (82, 83), (83, 84), (84, 85), (86, 88), (89, 90

In [21]:
print(s)
print(t)
print(om)
print(n)

'시설'이 양식생물을 의미하는 것과는 달리, 철주, 표지목과 같은 어구시설물을 의미한다고 풀이함이 상당하다
["'", '시설', "'", '이', '양식', '##생', '##물', '##을', '의미', '##하', '##는', '것', '##과', '##는', '달리', ',', '철', '##주', ',', '표지', '##목', '##과', '같', '##은', '어구', '##시설', '##물', '##을', '의미', '##한다', '##고', '풀이', '##함', '##이', '상당', '##하', '##다']
[(0, 0), (0, 1), (1, 3), (3, 4), (4, 5), (6, 8), (8, 9), (9, 10), (10, 11), (12, 14), (14, 15), (15, 16), (17, 18), (18, 19), (19, 20), (21, 23), (23, 24), (25, 26), (26, 27), (27, 28), (29, 31), (31, 32), (32, 33), (34, 35), (35, 36), (37, 39), (39, 41), (41, 42), (42, 43), (44, 46), (46, 48), (48, 49), (50, 52), (52, 53), (53, 54), (55, 57), (57, 58), (58, 59), (0, 0)]
{}


In [22]:
# for text, ne, offset_mapping in zip(texts, nes, offset_mappings):
#     print(text)
#     print(ne)
#     print(offset_mapping)
#     s = text
#     n = ne
#     om = offset_mapping
#     break

In [23]:
def get_char_labels(sentence, ne, label_to_index, normalize=False):
    """
    sentence: 일본인가 어딘가 간다고
    ne: {1: {'form': '정호성', 'label': 'PS_NAME', 'begin': 7, 'end': 10}, 2: {'form': '비서관', 'label': 'CV_POSITION', 'begin': 11, 'end': 14}}

    Labeling tags by character-level.
    Parameter "normalize" for KoBERTTokenizer, which used to normalize unicode text before tokenizing.
    """
    char_labels = [0] * len(sentence) # sentence의 글자 개수만큼 0으로 채워진 리스트
    # print(sentence) # 그리고 인제 정호성 비서관이
    # print(char_labels) # [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    for i in range(1, len(ne)+1): # ne의 인덱스가 1부터 시작하므로 +1 하는 것
        begin = ne[i]['begin'] # 엔티티가 시작하는 문장 인덱스
        end = ne[i]['end'] # 엔티티가 끝나는 문장 인덱스
        # print(begin, end)
        
        B_label = 'B-'+ne[i]['label'][:2] # PS_NAME -> B-PS
        I_label = 'I-'+ne[i]['label'][:2] # PS_NAME -> I-PS
        # print(B_label, I_label)

        for j in range(begin, end):
            char_labels[j] = label_to_index[I_label]
            # char_labels의 begin 부터 end 인덱스까지 I_label에 해당하는 번호로 업데이트 한다.
            # begin = 7, end = 10이고, I_label = I-PS일 때
            # [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]를
            # [0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0]로 변형
        # print(char_labels)

        char_labels[begin] = label_to_index[B_label]
        # 그리고 begin에 해당하는 인덱스만 B_label의 번호로 업데이트 한다.
        # [0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0]를
        # [0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0]로 변경
        # print(char_labels)
    # 이 과정을 각각의 엔티티에 대해서 수행하며 char_labels를 업데이트 해나간다.

    if normalize:
        normalized_sentence = unicodedata.normalize("NFKC", sentence)
        for i, char in enumerate(sentence):
            normalized_char = unicodedata.normalize("NFKC", char)
            if len(normalized_char) > 1:
                char_labels[i:i] = [char_labels[i]] * (len(normalized_char)-1)
                if len(char_labels) == len(normalized_sentence):
                    break        

    return char_labels

In [24]:
cl = get_char_labels(s, n, label_to_index, False)

In [25]:
print(cl)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [26]:
def get_token_labels(char_labels, offset_mappings):
    """
    char_labels = [0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 13, 14, 14, 0]
    offset_mappings = [(0, 0), (0, 3), (4, 6), (7, 9), (9, 10), (11, 14), (14, 15), (0, 0)]
    Labeling tags by token-level.
    """
    token_labels = []

    for offset_mapping in offset_mappings:
        start_offset, end_offset = offset_mapping
        if start_offset == end_offset:# (0, 0) 특수 토큰이면
            token_labels.append(-100) # -100을 리스트에 넣는다.
            continue
        token_labels.append(char_labels[start_offset])
        # print(char_labels[start_offset])

    return token_labels

In [27]:
tl = get_token_labels(cl, om)

In [28]:
print(tl)

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [29]:
# 맨 앞에 '[CLS]' 추가
t.insert(0, '[CLS]')
# 맨 뒤에 '[SEP]' 추가
t.append('[SEP]')
print(t)

['[CLS]', "'", '시설', "'", '이', '양식', '##생', '##물', '##을', '의미', '##하', '##는', '것', '##과', '##는', '달리', ',', '철', '##주', ',', '표지', '##목', '##과', '같', '##은', '어구', '##시설', '##물', '##을', '의미', '##한다', '##고', '풀이', '##함', '##이', '상당', '##하', '##다', '[SEP]']


In [30]:
for token, entity_idx in zip(t, tl):
    if entity_idx == -100:
        continue
    else:
        label = index_to_label[entity_idx]
        print(token.ljust(5), label.rjust(5))

'         O
시설        O
'         O
이         O
양식        O
##생       O
##물       O
##을       O
의미        O
##하       O
##는       O
것         O
##과       O
##는       O
달리        O
,         O
철         O
##주       O
,         O
표지        O
##목       O
##과       O
같         O
##은       O
어구        O
##시설      O
##물       O
##을       O
의미        O
##한다      O
##고       O
풀이        O
##함       O
##이       O
상당        O
##하       O
##다       O


In [31]:
for text, ne, offset_mapping in zip(texts, nes, offset_mappings):
    char_labels = get_char_labels(text, ne, label_to_index, normalize=text_normalize)
    # 오늘 왜 머리 올리고 왔어?
    # [15, 16, 0, 0, 0, 23, 24, 0, 0, 0, 0, 0, 0, 0, 0]
    token_labels = get_token_labels(char_labels, offset_mapping)
    # [-100, 15, 0, 23, 0, 0, 0, 0, 0, -100]
    ne_ids.append(token_labels)

print("Sequence labeling completed : |labels| %d" % (len(ne_ids)))

Sequence labeling completed : |labels| 177


In [32]:
return_data = pd.DataFrame([encoded["input_ids"], encoded["attention_mask"], ne_ids, data['sentence_class'].values], index=[
                            "input_ids", "attention_mask", "labels", "sentence_class"]).T
if config.with_text:
    return_data['sentence'] = texts

In [33]:
return_data

,input_ids,attention_mask,labels,sentence_class
0,"[0, 1545, 2133, 2252, 6233, 544, 7957, 7488, 8...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 15, 16, 16, 0, 0, 0, 0, 17, 18, 18, 18,...",JB
1,"[0, 21, 18, 7577, 2522, 2079, 3734, 2170, 8305...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 17, 0, 0, 0, 0, 17, 0, 0, 0, 0, 0...",JB
2,"[0, 11, 3953, 11, 1504, 6277, 2065, 2266, 2069...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Out
3,"[0, 1517, 2200, 5636, 4615, 2496, 2015, 1537, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Out
4,"[0, 24673, 2066, 2079, 4913, 2200, 5183, 2897,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 9, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",NM
...,...,...,...,...
172,"[0, 1485, 12, 7864, 12607, 13, 3723, 8716, 217...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",JB
173,"[0, 1513, 2051, 3983, 12489, 2207, 2079, 3777,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Out
174,"[0, 6954, 28, 2079, 544, 3951, 8105, 1116, 149...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",RL
175,"[0, 3995, 16, 558, 2361, 2072, 1943, 2079, 776...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 25, 26, 26, 0, 0, ...",BP


In [34]:
label_info = {
    "label_list": label_list,
    "label_to_index": label_to_index,
    "index_to_label": index_to_label
}

return_values = {
    "data": return_data.to_dict(),
    "label_info": label_info,
    "pad_token": (tokenizer.pad_token, tokenizer.pad_token_id),
    "pretrained_model_name": tokenizer.name_or_path
}

save_path = config.save_path   
fn = os.path.split(config.load_fn)[1].split('.')[0]
plm_name = pretrained_model_name.replace('/', '_')
save_fn = os.path.join(save_path, f'{fn}.{plm_name}.encoded.pickle')

with open(save_fn, "wb") as f:
    pickle.dump(return_values, f, 4)
print("Encoded data saved as %s " % save_fn)

Encoded data saved as data/encoded/train.klue_roberta-base.encoded.pickle 
